In [1]:
# unpacking json file

import pandas as pd
import json

In [2]:
with open('prem_2024_data.json') as f:
    d = json.load(f)
       

In [3]:
len(d['lineups']) #['0'][0]

349

In [ ]:
fixture_info_list = []

for i in range(len(d['fixture'])):
    index = str(i)

    fixture_info = {
    "fixture_id": d['fixture'][index]['id'],
    "country": d['league'][index]['country'],
    "competition": d['league'][index]['name'],
    "home_team": d['teams'][index]['home']['name'],
    "away_team": d['teams'][index]['away']['name'],
    "home_formation": d['lineups'][index][0]['formation'],
    "away_formation": d['lineups'][index][1]['formation'],
    "home_start_xi" : d['lineups'][index][0]['startXI'],
    "away_start_xi": d['lineups'][index][1]['startXI'],
    "home_subs": d['lineups'][index][0]['substitutes'],
    "away_subs": d['lineups'][index][0]['substitutes']
    }

    fixture_info_list.append(fixture_info)

df = pd.DataFrame(fixture_info_list)

# Step 1: Combine starters and subs into full squads
df['home_squad'] = df['home_start_xi'] + df['home_subs']
df['away_squad'] = df['away_start_xi'] + df['away_subs']

# Step 2: Loop through fixtures and build rows
squad_rows = []

for _, row in df.iterrows():
    fixture_id = row['fixture_id']
    home_team = row['home_team']
    away_team = row['away_team']
    competition = row['competition']
    home_formation = row['home_formation']
    away_formation = row['away_formation']

    home_squad = row['home_squad']
    away_squad = row['away_squad']
    
    max_len = max(len(home_squad), len(away_squad))
    
    for i in range(max_len):
        home_player = home_squad[i] if i < len(home_squad) else {}
        away_player = away_squad[i] if i < len(away_squad) else {}

        squad_rows.append({
            'fixture_id': fixture_id,
            'competition': competition,
            'player_slot': i + 1,
            'home_team': home_team,
            'home_formation': home_formation,
            'home_player_name': home_player.get('player', {}).get('name'),
            'home_player_number': home_player.get('player', {}).get('number'),
            'home_player_pos': home_player.get('player', {}).get('pos'),
            'home_player_grid': home_player.get('player', {}).get('grid'),
            'away_team': away_team,
            'away_formation': away_formation,
            'away_player_name': away_player.get('player', {}).get('name'),
            'away_player_number': away_player.get('player', {}).get('number'),
            'away_player_pos': away_player.get('player', {}).get('pos'),
            'away_player_grid': away_player.get('player', {}).get('grid'),
        })

# Step 3: Final squad-level DataFrame
df_squads = pd.DataFrame(squad_rows)


In [26]:

rows = []
# assume d['fixture'], d['league'], d['teams'] & d['lineups'] are all dicts keyed by the same strings
for key, fix in d['fixture'].items():
    league = d['league'][key]
    teams  = d['teams'][key]
    lineups = d['lineups'][key]  # [ home_lineup, away_lineup ]

    for side, lineup in zip(['home','away'], lineups):
        full_squad = lineup['startXI'] + lineup['substitutes']
        for slot, pl in enumerate(full_squad, start=1):
            rows.append({
                'fixture_id':   fix['id'],
                'country':      league['country'],
                'competition':  league['name'],
                'team_side':    side,                    # 'home' or 'away'
                'team_name':    teams[side]['name'],
                'formation':    lineup['formation'],
                'player_slot':  slot,
                'player_name':  pl['player']['name'],
                'player_number':pl['player']['number'],
                'player_pos':   pl['player']['pos'],
                'player_grid':  pl['player']['grid']
            })

df_squads = pd.DataFrame(rows)

In [27]:
df_squads

,fixture_id,country,competition,team_side,team_name,formation,player_slot,player_name,player_number,player_pos,player_grid
0,1208021,England,Premier League,home,Manchester United,4-2-3-1,1,A. Onana,24,G,1:1
1,1208021,England,Premier League,home,Manchester United,4-2-3-1,2,N. Mazraoui,3,D,2:4
2,1208021,England,Premier League,home,Manchester United,4-2-3-1,3,H. Maguire,5,D,2:3
3,1208021,England,Premier League,home,Manchester United,4-2-3-1,4,Lisandro Martínez,6,D,2:2
4,1208021,England,Premier League,home,Manchester United,4-2-3-1,5,Diogo Dalot,20,D,2:1
...,...,...,...,...,...,...,...,...,...,...,...
13945,1208372,England,Premier League,away,Tottenham,4-3-3,16,Cristian Romero,17,D,None
13946,1208372,England,Premier League,away,Tottenham,4-3-3,17,Micky van de Ven,37,D,None
13947,1208372,England,Premier League,away,Tottenham,4-3-3,18,Rodrigo Bentancur,30,M,None
13948,1208372,England,Premier League,away,Tottenham,4-3-3,19,Brennan Johnson,22,F,None


In [28]:
df_squads.to_csv("data/data/fbref_dashboard/prem_24_lineups_long.csv")


In [24]:
import numpy as np
X_MIN = 10
Y_MIN = 10
X_MAX = 50
Y_MAX = 70

def players_plotting_coords(df, home_team, away_team):

    tidy = df[
    (df['away_team'] == away_team) &
    (df['home_team'] == home_team) &
    (df['home_player_grid'].str.contains(":", na=False))
    ].copy()
    
    tidy[["xplot","yplot"]] = tidy["home_player_grid"].str.split(':', expand = True)
    tidy['xplot'] = tidy['xplot'].astype(float)
    tidy['yplot'] = tidy['yplot'].astype(float)
    tidy['home_formation_list'] = tidy['home_formation'].str.split('-').to_list()
    tidy['xplot'] = X_MIN + (tidy['xplot'] - 1) * (X_MAX - X_MIN) / tidy['home_formation_list'].apply(len)
    # Calculate max yplot per xplot
    max_y_per_x = tidy.groupby("xplot")["yplot"].transform("max")

    # Apply scaling using per-x max values
    tidy['yplot'] = np.where(
        tidy['home_player_pos'] == "G",
        40,  # fixed value for goalkeeper
        Y_MIN + ((tidy['yplot'] - 1) * (Y_MAX - Y_MIN) / (max_y_per_x-1) )
    )
    return tidy

output_df = players_plotting_coords(df_squads, "Crystal Palace","Fulham")

In [25]:
output_df

,fixture_id,competition,player_slot,home_team,home_formation,home_player_name,home_player_number,home_player_pos,home_player_grid,away_team,away_formation,away_player_name,away_player_number,away_player_pos,away_player_grid,xplot,yplot,home_formation_list
2057,1208126,Premier League,1,Crystal Palace,3-4-2-1,D. Henderson,1,G,1:1,Fulham,4-2-3-1,B. Leno,1,G,1:1,10.0,40.0,"[3, 4, 2, 1]"
2058,1208126,Premier League,2,Crystal Palace,3-4-2-1,T. Chalobah,27,D,2:3,Fulham,4-2-3-1,K. Tete,2,D,2:4,20.0,70.0,"[3, 4, 2, 1]"
2059,1208126,Premier League,3,Crystal Palace,3-4-2-1,M. Lacroix,5,D,2:2,Fulham,4-2-3-1,J. Andersen,5,D,2:3,20.0,40.0,"[3, 4, 2, 1]"
2060,1208126,Premier League,4,Crystal Palace,3-4-2-1,N. Clyne,17,D,2:1,Fulham,4-2-3-1,C. Bassey,3,D,2:2,20.0,10.0,"[3, 4, 2, 1]"
2061,1208126,Premier League,5,Crystal Palace,3-4-2-1,D. Muñoz,12,M,3:4,Fulham,4-2-3-1,A. Robinson,33,D,2:1,30.0,70.0,"[3, 4, 2, 1]"
2062,1208126,Premier League,6,Crystal Palace,3-4-2-1,M. Guéhi,6,M,3:3,Fulham,4-2-3-1,Andreas Pereira,18,M,3:2,30.0,50.0,"[3, 4, 2, 1]"
2063,1208126,Premier League,7,Crystal Palace,3-4-2-1,J. Devenny,55,M,3:2,Fulham,4-2-3-1,S. Berge,16,M,3:1,30.0,30.0,"[3, 4, 2, 1]"
2064,1208126,Premier League,8,Crystal Palace,3-4-2-1,T. Mitchell,3,M,3:1,Fulham,4-2-3-1,A. Iwobi,17,M,4:3,30.0,10.0,"[3, 4, 2, 1]"
2065,1208126,Premier League,9,Crystal Palace,3-4-2-1,D. Kamada,18,F,4:2,Fulham,4-2-3-1,E. Smith Rowe,32,M,4:2,40.0,70.0,"[3, 4, 2, 1]"
2066,1208126,Premier League,10,Crystal Palace,3-4-2-1,I. Sarr,7,F,4:1,Fulham,4-2-3-1,R. Nelson,19,M,4:1,40.0,10.0,"[3, 4, 2, 1]"


In [54]:
fixture_df = pd.DataFrame(d['fixture'])
fixture_df_melt = fixture_df.melt()


In [58]:
fixture_df_melt.iloc[6]

AttributeError: 'Series' object has no attribute 'json'

In [49]:
lineup_df = pd.DataFrame(d['lineups'])

In [52]:
lineup_df.melt()

,variable,value
0,0,"{'team': {'id': 33, 'name': 'Manchester United..."
1,0,"{'team': {'id': 36, 'name': 'Fulham', 'logo': ..."
2,1,"{'team': {'id': 57, 'name': 'Ipswich', 'logo':..."
3,1,"{'team': {'id': 40, 'name': 'Liverpool', 'logo..."
4,2,"{'team': {'id': 42, 'name': 'Arsenal', 'logo':..."
...,...,...
693,346,"{'team': {'id': 41, 'name': 'Southampton', 'lo..."
694,347,"{'team': {'id': 50, 'name': 'Manchester City',..."
695,347,"{'team': {'id': 39, 'name': 'Wolves', 'logo': ..."
696,348,"{'team': {'id': 48, 'name': 'West Ham', 'logo'..."


In [34]:
test_df = test_df.melt()

In [24]:
test_2 = pd.json_normalize(test_df['value'])

In [25]:
test_2

,formation,startXI,substitutes,team.id,team.name,team.logo,team.colors.player.primary,team.colors.player.number,team.colors.player.border,team.colors.goalkeeper.primary,team.colors.goalkeeper.number,team.colors.goalkeeper.border,coach.id,coach.name,coach.photo
0,4-2-3-1,"[{'player': {'id': 526, 'name': 'A. Onana', 'n...","[{'player': {'id': 284324, 'name': 'A. Garnach...",33,Manchester United,https://media.api-sports.io/football/teams/33.png,ea0000,ffffff,ea0000,000000,ffffff,000000,1993.0,E. ten Hag,https://media.api-sports.io/football/coachs/19...
1,4-2-3-1,"[{'player': {'id': 1438, 'name': 'B. Leno', 'n...","[{'player': {'id': 19025, 'name': 'T. Cairney'...",36,Fulham,https://media.api-sports.io/football/teams/36.png,ffffff,000000,ffffff,d01e2a,ffffff,d01e2a,10.0,Marco Silva,https://media.api-sports.io/football/coachs/10...
2,4-2-3-1,"[{'player': {'id': 19541, 'name': 'C. Walton',...","[{'player': {'id': 18823, 'name': 'B. Johnson'...",57,Ipswich,https://media.api-sports.io/football/teams/57.png,0055aa,ffffff,0055aa,0f0f0e,000000,0f0f0e,16556.0,K. McKenna,https://media.api-sports.io/football/coachs/16...
3,4-2-3-1,"[{'player': {'id': 280, 'name': 'Alisson Becke...","[{'player': {'id': 1145, 'name': 'I. Konaté', ...",40,Liverpool,https://media.api-sports.io/football/teams/40.png,e41e2c,ffffff,e41e2c,23262b,f3f5f0,23262b,2006.0,A. Slot,https://media.api-sports.io/football/coachs/20...
4,4-3-3,"[{'player': {'id': 19465, 'name': 'David Raya'...","[{'player': {'id': 38746, 'name': 'J. Timber',...",42,Arsenal,https://media.api-sports.io/football/teams/42.png,e10000,ffffff,e10000,00d5ff,ffffff,00d5ff,7248.0,Mikel Arteta,https://media.api-sports.io/football/coachs/72...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
693,3-4-2-1,"[{'player': {'id': 20355, 'name': 'Aaron Ramsd...","[{'player': {'id': 304317, 'name': 'Tyler Dibl...",41,Southampton,https://media.api-sports.io/football/teams/41.png,c4ea87,000000,c4ea87,affd69,ffffff,affd69,600.0,S. Rusk,https://media.api-sports.io/football/coachs/60...
694,4-2-3-1,"[{'player': {'id': 617, 'name': 'Ederson', 'nu...","[{'player': {'id': 5, 'name': 'Manuel Akanji',...",50,Manchester City,https://media.api-sports.io/football/teams/50.png,abd1f5,ffffff,abd1f5,ee4665,070707,ee4665,NaN,None,NaN
695,3-4-2-1,"[{'player': {'id': 1590, 'name': 'José Sá', 'n...","[{'player': {'id': 2056, 'name': 'Pablo Sarabi...",39,Wolves,https://media.api-sports.io/football/teams/39.png,fecd32,000000,fecd32,00c32b,ffffff,00c32b,NaN,None,NaN
696,3-4-2-1,"[{'player': {'id': 253, 'name': 'Alphonse Aréo...","[{'player': {'id': 1231, 'name': 'Vladimír Cou...",48,West Ham,https://media.api-sports.io/football/teams/48.png,74112e,ffffff,74112e,d400ff,ffffff,d400ff,12.0,G. Potter,https://media.api-sports.io/football/coachs/12...


In [ ]:
import pandas as pd

# assume test_df has columns: team, coach, formation, startXI, substitutes

def explode_role(df, list_col, role_name):
    # 1. explode the chosen list
    exploded = df.explode(list_col).reset_index(drop=True)
    # 2. flatten team & coach
    team_flat  = pd.json_normalize(exploded["team.name"]).add_prefix("team_")
    coach_flat = pd.json_normalize(exploded["coach.name"]).add_prefix("coach_")
    # 3. flatten the player dict
    play_flat  = pd.json_normalize(exploded[list_col]).add_prefix("player_")
    # 4. stitch back together, drop old columns, tag role
    base = exploded.drop(columns=["team.name","coach.name", list_col])
    out  = pd.concat([base, team_flat, coach_flat, play_flat], axis=1)
    out["role"] = role_name
    return out

# build two tables…
starters = explode_role(test_2, "startXI",    role_name="starter")
subs     = explode_role(test_2, "substitutes", role_name="sub")

# …and glue them into one
players = pd.concat([starters, subs], ignore_index=True)

In [29]:
players.head(50)

,formation,substitutes,team.id,team.logo,team.colors.player.primary,team.colors.player.number,team.colors.player.border,team.colors.goalkeeper.primary,team.colors.goalkeeper.number,team.colors.goalkeeper.border,coach.id,coach.photo,player_player.id,player_player.name,player_player.number,player_player.pos,player_player.grid,role,startXI
0,4-2-3-1,"[{'player': {'id': 284324, 'name': 'A. Garnach...",33,https://media.api-sports.io/football/teams/33.png,ea0000,ffffff,ea0000,000000,ffffff,000000,1993.0,https://media.api-sports.io/football/coachs/19...,526,A. Onana,24,G,1:1,starter,NaN
1,4-2-3-1,"[{'player': {'id': 284324, 'name': 'A. Garnach...",33,https://media.api-sports.io/football/teams/33.png,ea0000,ffffff,ea0000,000000,ffffff,000000,1993.0,https://media.api-sports.io/football/coachs/19...,545,N. Mazraoui,3,D,2:4,starter,NaN
2,4-2-3-1,"[{'player': {'id': 284324, 'name': 'A. Garnach...",33,https://media.api-sports.io/football/teams/33.png,ea0000,ffffff,ea0000,000000,ffffff,000000,1993.0,https://media.api-sports.io/football/coachs/19...,2935,H. Maguire,5,D,2:3,starter,NaN
3,4-2-3-1,"[{'player': {'id': 284324, 'name': 'A. Garnach...",33,https://media.api-sports.io/football/teams/33.png,ea0000,ffffff,ea0000,000000,ffffff,000000,1993.0,https://media.api-sports.io/football/coachs/19...,2467,Lisandro Martínez,6,D,2:2,starter,NaN
4,4-2-3-1,"[{'player': {'id': 284324, 'name': 'A. Garnach...",33,https://media.api-sports.io/football/teams/33.png,ea0000,ffffff,ea0000,000000,ffffff,000000,1993.0,https://media.api-sports.io/football/coachs/19...,886,Diogo Dalot,20,D,2:1,starter,NaN
5,4-2-3-1,"[{'player': {'id': 284324, 'name': 'A. Garnach...",33,https://media.api-sports.io/football/teams/33.png,ea0000,ffffff,ea0000,000000,ffffff,000000,1993.0,https://media.api-sports.io/football/coachs/19...,747,Casemiro,18,M,3:2,starter,NaN
6,4-2-3-1,"[{'player': {'id': 284324, 'name': 'A. Garnach...",33,https://media.api-sports.io/football/teams/33.png,ea0000,ffffff,ea0000,000000,ffffff,000000,1993.0,https://media.api-sports.io/football/coachs/19...,284322,K. Mainoo,37,M,3:1,starter,NaN
7,4-2-3-1,"[{'player': {'id': 284324, 'name': 'A. Garnach...",33,https://media.api-sports.io/football/teams/33.png,ea0000,ffffff,ea0000,000000,ffffff,000000,1993.0,https://media.api-sports.io/football/coachs/19...,157997,A. Diallo,16,M,4:3,starter,NaN
8,4-2-3-1,"[{'player': {'id': 284324, 'name': 'A. Garnach...",33,https://media.api-sports.io/football/teams/33.png,ea0000,ffffff,ea0000,000000,ffffff,000000,1993.0,https://media.api-sports.io/football/coachs/19...,19220,M. Mount,7,M,4:2,starter,NaN
9,4-2-3-1,"[{'player': {'id': 284324, 'name': 'A. Garnach...",33,https://media.api-sports.io/football/teams/33.png,ea0000,ffffff,ea0000,000000,ffffff,000000,1993.0,https://media.api-sports.io/football/coachs/19...,909,M. Rashford,10,M,4:1,starter,NaN
